## Ising Model GAN

In [1]:
!wget https://raw.githubusercontent.com/pSzyc/Ising/main/Model/GAN/model.py
!wget https://raw.githubusercontent.com/pSzyc/Ising/main/Model/input_pipeline.py

--2023-11-25 12:24:37--  https://raw.githubusercontent.com/pSzyc/Ising/main/Model/GAN/model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2506 (2.4K) [text/plain]
Saving to: ‘model.py’

model.py            100%[===================>]   2.45K  --.-KB/s    in 0s      

2023-11-25 12:24:38 (49.9 MB/s) - ‘model.py’ saved [2506/2506]

--2023-11-25 12:24:38--  https://raw.githubusercontent.com/pSzyc/Ising/main/Model/input_pipeline.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2226 (2.2K) [text/plain]
Saving to: ‘input_pipeline.

In [ ]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
import time
from IPython import display
from input_pipeline import dataset_colab_pipeline
from model import make_discriminator_model, make_generator_model, generator_loss, discriminator_loss
import csv
from google.colab import drive
drive.mount('/content/drive')
data_path = "drive/MyDrive/Licencjat"

## Load data

In [ ]:
trainset_path = f"{data_path}/Trainset.npy"
testset_path =  f"{data_path}/Testset.npy"
batch_size = 32
train_set = dataset_colab_pipeline(trainset_path, flatten=False, batch_size=batch_size)
test_set = dataset_colab_pipeline(testset_path, flatten=False, batch_size=batch_size)

## Model

In [ ]:
generator = make_generator_model()
discriminator = make_discriminator_model()

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

## Setup training

In [ ]:
noise_dim = 100
num_examples_to_generate = 16

# We'll re-use this random vector used to seed the generator so
# it will be easier to see the improvement over time.
random_vector_for_generation = tf.random.normal([num_examples_to_generate,
                                                 noise_dim])

In [ ]:
def train_step(images, gen_loss_log, disc_loss_log):
      noise = tf.random.normal([batch_size, noise_dim])

      with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        generated_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(generated_output)
        gen_loss_log.append(gen_loss)
        disc_loss = discriminator_loss(real_output, generated_output)
        disc_loss_log.append(disc_loss)

      gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  predictions = tf.round(model(test_input, training=False))

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(np.rint(predictions[i, :, :, 0]))
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

In [ ]:
def train(dataset, epochs, gen_loss_log, disc_loss_log):

  for epoch in range(epochs):
    start = time.time()

    for images in tqdm(dataset): train_step(images, gen_loss_log, disc_loss_log)

    display.clear_output(wait=True)
    generate_and_save_images(
      generator,
      epoch + 1,
      random_vector_for_generation
    )
    print (f"Time taken for epoch {epoch} is {time.time()- start} sec")

## Train the GAN!

In [ ]:
EPOCHS=30
gen_loss_log=[]
disc_loss_log=[]

In [ ]:
%%time
train(train_set, EPOCHS,gen_loss_log,disc_loss_log)

## Plot the loss of the generator and discriminator

In [ ]:
plt.plot(np.asarray(gen_loss_log))
plt.plot(np.asarray(disc_loss_log))

In [ ]:
predictions = generator(random_vector_for_generation, training=False)

In [ ]:
for data in test_set.take(1):
  sns.heatmap(data[0, :, :, 0])

In [ ]:
import seaborn as sns
sns.heatmap(predictions[0, :, :, 0]>0)
